# Joint conf-call report #4

In [1]:
import datetime
print(str(datetime.datetime.today()))

2018-05-28 14:46:22.329854


# Velocity approximation
As it isn't actually a lower bound. We suppose by the time node $v$ finishes computation for iteration $t$ then its dependencies in turn have already received the required informations from their dependencies so they can start calculations for iteration $t$.

The time taken by node $v$ is a random variable $X$ while its $k$ (e.g. the graph's degree) dependencies are I.I.D. $X_1,...,X_k$. Let $Y=max(X_1,...,X_k)$ be the slowest dependency node to complete the task,

$$Pr(Y \leq y) = \prod_{i=1}^{k} Pr(X_i \leq y) = Pr(X \leq y)^k = F_X(y)^k$$

$$\mathbb{E}[Y] = \int_{0}^{\infty} (1-Pr(Y\leq y))\ dy = \int_{0}^{\infty} (1-F_X(y)^k)\ dy.$$

The total time $v$ has to wait is $\mathbb{E}[Y]+\mathbb{E}[X]$, so the single iteration velocity can be expressed as

$$V = \frac{1}{\mathbb{E}[Y]+\mathbb{E}[X]}.$$

# Velocity assuming different time distributions

## Exponential time
The one always used up to now.

$$\mathbb{E}[Y] = \int_{0}^{\infty} 1-(1-e^{-\lambda y})^k\ dy$$

$$V = \frac{\lambda}{1+\sum_{i=1}^{k} \frac{1}{i}}$$

## Uniform time
$Y = max(X_1,...,X_k)$ with $X \sim \mathcal{U}(a,b)$.

Uniform's CDF:
$$
F_X(x) =
\begin{cases}
    0 & \text{if } x < a \\
    \frac{x-a}{b-a} & \text{if } a \leq x < b \\
    1 & \text{if } x \geq b \end{cases}
$$
<br>

$$
\begin{align}
\mathbb{E}[Y] = \int_{0}^{\infty} 1 - (F_X(y)^k) dy &= \int_{0}^{a} 1\ dy + \int_{a}^{b} 1-\left(\frac{y-a}{b-a}\right)^k dy + \int_{b}^{\infty} 1-1\ dy =\\
&= a + \int_{a}^{b} 1\ dy - \frac{1}{(b-a)^k} \int_{a}^{b} (y-a)^k dy + 0 =\\
&= b - \frac{1}{(b-a)^k} \left[\frac{(y-a)^{k+1}}{k+1}\right]_{a}^{b} =\\
&= b - \frac{1}{(b-a)^k} \frac{(b-a)^{k+1}}{k+1} =\\
&= b + \frac{a-b}{k+1} = \frac{kb+b-b+a}{k+1} =\\
&= \frac{a+kb}{k+1}
\end{align}
$$
<br>

$$
\overline{T} = \mathbb{E}[X] + \mathbb{E}[Y] = \frac{a+b}{2} + \frac{a+kb}{k+1} = \frac{a(k+3) + b(3k+1)}{2(k+1)}
$$
<br>

$$
\overline{V} = \frac{1}{\overline{T}} = \frac{1}{\mathbb{E}[X] + \mathbb{E}[Y]} = \frac{2(k+1)}{a(k+3) + b(3k+1)}.
$$

For $a=0$ and $b=2$
$$
\overline{T} = \frac{3k+1}{k+1}.
$$

## Type II Pareto time
Type II Pareto's CDF:

$$
F_X(x) = 1-\left(1-\frac{x}{\lambda}\right)^{-\alpha}
$$

$Y = max(X_1,...,X_k)$ with $X \sim Pareto(\alpha)$.

$$
\begin{equation}
\mathbb{E}[Y] = \int_{0}^{\infty} 1 - (F_X(y)^k)\ dy = \dots
\end{equation}
$$

*to be completed...*

# Residual time CDF for non-memoryless distributions

$$F_{Xres}(x) = \frac{1}{\mathbb{E}[X]} \int_{0}^{x} (1-F_x(t))\ dt$$

## Uniform with residual time CDF

Uniform's residual time CDF:
$$
\begin{equation}
    F_{Xres}(x) = \frac{1}{\mathbb{E}[X]} \int_{0}^{x} (1-F_x(t))\ dt = \frac{2}{a+b} \left[ x-\int_{0}^{x} F_X(t)\ dt \right]
\end{equation}
$$

$$
F_{Xres}(x) = \frac{2}{a+b} 
\begin{cases}
    x & \text{if } x \leq a \\
    x-\int_{a}^{x}\frac{t-a}{b-a}\ dt & \text{if } a < x < b\\
    x-\int_{a}^{b}\frac{t-a}{b-a}\ dt -(x-b)& \text{if } x \geq b
\end{cases}
$$

$$
\begin{cases}
    \int_{a}^{x} \frac{t-a}{b-a}\ dt\\
    u = \frac{t-a}{b-a}
\end{cases}
$$

$$
\begin{cases}
    t = u(b-a)+a\\
    dt = (b-a)\ du
\end{cases}
$$

$$
\begin{cases}
    t = a \Rightarrow u=0\\
    t = x \Rightarrow u = \frac{x-a}{b-a}
\end{cases}
$$

$$
\int_{a}^{x} \frac{t-a}{b-a}\ dt = \int_{0}^{\frac{x-a}{b-a}} u(b-a)\ du = (b-a) \frac{u^2}{2}\Biggr|_0^{\frac{x-a}{b-a}} = \frac{(x-a)^2}{2(b-a)}
$$

So


$$
F_{Xres}(x) = 
\begin{cases}
    \frac{2x}{a+b} & \text{if } x \leq a \\
    \frac{x^2-2bx+a^2}{a^2-b^2} & \text{if } a < x < b\\
    1 & \text{if } x \geq b
\end{cases}
$$


$Yres = max(Xres_1,...,Xres_k)$

$$
\begin{align}
\mathbb{E}[Yres] & = \int_{0}^{+\infty}1-F_{Xres}(y)^k\ dy =\\
& = \int_{0}^{a} 1-\left(\frac{2y}{a+b}\right)^k\ dy + \int_{a}^{b} 1- \left(\frac{y^2-2by+a^2}{a^2-b^2}\right)^k\ dy + \int_{b}^{+\infty}1-1\ dy =\\
& = a - \frac{2^k}{(a+b)^k} \int_{0}^{a} y^k\ dy +b-a - \frac{1}{(a^2-b^2)^k} \int_{a}^{b} (y^2-2by+a^2)^k\ dy=\\
& = b - \frac{2^k a^{k+1}}{(a+b)^k (k+1)} - \frac{1}{(a^2-b^2)^k} \int_{a}^{b} (y^2-2by+a^2)^k\ dy.
\end{align}
$$

For integral $\int_{a}^{b} (y^2-2by+a^2)^k\ dy$ (with generic $k\geq 1$) there are no primitives to calculate its closed form while it is always possible for fixed $k$.

For $a=0,\ b=2,\ k=1$ (cycle):
$$\mathbb{E}[Yres] = b - \frac{2^k a^{k+1}}{(a+b)^k (k+1)} - \frac{1}{(a^2-b^2)^k} \int_{a}^{b} (y^2-2by+a^2)^k\ dy = 2+\frac{1}{4}\int_{0}^{2} t^2 -4t\ dt = 2 + \frac{1}{4} \left(-\frac{16}{3}\right) = \frac{2}{3}$$

and velocity

$$\overline{V} = \frac{1}{\mathbb{E}[Yres] + \mathbb{E}[X]} = \frac{1}{\frac{2}{3} + 1} = \frac{3}{5}$$

## Type II Pareto with residual time
Type II Pareto CDF with parameters $\alpha$ and $\sigma$:
$$F_X(x) = 1-\left(1+\frac{x}{\sigma}\right)^{-\alpha}.$$

$$
\begin{align}
F_{Xres}(x) &= \frac{1}{\mathbb{E}[X]} \int_{0}^{x} \left(1+\frac{u}{\sigma}\right)^{-\alpha}\ du = \qquad \left(\text{with } 1+\frac{u}{\sigma}=\nu\right) \\
&= \frac{1}{\mathbb{E}[X]} \int_{1}^{1+\frac{x}{\sigma}} \sigma \nu^{-\alpha}\ d\nu \\
&= \frac{1}{\mathbb{E}[X]} \sigma \frac{\nu^{-\alpha+1}}{\alpha-1}\Biggr|_{1+\frac{x}{\sigma}}^{1} =\\
&= \frac{\sigma}{(\alpha-1)\mathbb{E}[X]}\left(1-\left(1+\frac{x}{\sigma}\right)^{-\alpha+1}\right)=\\
&= 1-\left(1+\frac{x}{\sigma}\right)^{-\alpha+1}.
\end{align}
$$

Type II Pareto Residual lifetime CDF is a classical type II Pareto with parameters $\alpha-1$ and $\sigma$.

$Yres = max(Xres_1,...,Xres_k)$.

$$
\begin{align}
\mathbb{E}[Yres] &= \int_{0}^{\infty} 1-\left(1-\left(1+\frac{y}{\sigma}\right)^{-\alpha+1}\right)^k\ dy =\\
&= \int_{0}^{\infty} \sum_{i=1}^{k} \left[\binom{k}{i} (-1)^{(i+1)} \left(1+\frac{y}{\sigma}\right)^{(-\alpha+1)i}\right]\ dy =\\
&= \sum_{i=1}^{k} \left[\binom{k}{i} (-1)^{(i+1)} \int_{0}^{\infty} \underbrace{\left(1+\frac{y}{\sigma}\right)^{(-\alpha+1)i}}_\text{Pareto II for $\sigma$ and $(\alpha-1)i$}\ dy\right] =\\
&= \sum_{i=1}^{k} \left[\binom{k}{i} (-1)^{(i+1)} \frac{\sigma}{(\alpha-1)i-1}\right].
\end{align}
$$

$$\overline{V}=\frac{1}{\mathbb{E}[Yres]+\mathbb{E}[X]} = \frac{1}{\sum_{i=1}^{k} \left[\binom{k}{i} (-1)^{(i+1)} \frac{\sigma}{(\alpha-1)i-1}\right] + \frac{\sigma}{\alpha-1}} = \frac{1}{\sum_{i=1}^{k} \left[\binom{k}{i} (-1)^{(i+1)} \frac{\sigma}{(\alpha-1)i-1}\right] + \frac{\sigma}{\alpha-1}}$$

**Example**: ($k=1$, e.g. Cycle).

$\mathbb{E}[Yres] = \frac{\sigma}{\alpha-2}$, $\mathbb{E}[X]= \frac{\sigma}{\alpha-1}$.

For $\alpha=3$, $\sigma=2$:

$$
\overline{V} = \frac{1}{2+1} = \frac{1}{3}
$$

---
# *Everything below this line is just a raw draft*
---

## A hypothetical real (but very pessimistic) lower bound $\forall k$

$$V_l = \frac{\lambda}{1 + \sum_{j=0}^{\lambda-1} \sum_{i=1}^{\frac{n}{\lambda}(\lambda-j)} \frac{1}{i}}$$

where $d$ is the graph's diameter.

Let $u$ be the most advanced node in computation and suppose it being at iteration $t$. *Remember "node $u$ is at iteration $t$" means that $u$ has already performed the $t$-th update of its local model, so it owns $\mathbf{w}^{(t)}_u$*.

Let $d(u,v)$ be the "directed" distance from $u$ to $v$, e.g. the minimum amount of directed edges one needs to cross in order to go from $u$ to $v$ (obviously edges can be crossed just in the proper direction), then:

- nodes at distance 1 from $u$ must be at least at iteration $t-1$;
- nodes at distance 2 from $u$ must be at least at iteration $t-2$;
- in general, nodes at distance $l : l \leq d$ from $u$ must be at least at iteration $t-l$.

Due to exponential memoryless property, all exponential variables that rule the duration of each node's iteration can be regenerated in any moment. Therefore is legit assuming that right now node $u$ is starting iteration $t$ and nodes at distance $l$ from $u$ are starting iteration $t-l$ for all $l \leq d$.

The worst case for node $u$ is the following:
- by the time node $u$ finishes iteration $t$ it doesn't own inputs from its parent to start iteration $t+1$. So, node $u$ has finished, all random exponential variables can be regenerated, $u$ have to wait for the slowest node among those at distance 1 from it;
- but nodes at distance 1 from $u$ cannot start iteration $t$ since their still waiting their parents for their outcomes (e.g. nodes at distance 2 from $u$ haven't performed iteration $t-1$ yet); the same for nodes at distance 2 from $u$: they are waiting their parents, and so on up to nodes at distance $d$ from $u$, a straggler belongs to them, it is slowing down the whole computation.

In this case, the time taken by the slowest node at distance $d$ is distributed as $max \{X_1,...,X_n\}$ (where $X_i \sim Exp(\lambda)$ and $n$ is the total amount of computational units) since it is . After waiting this time a chain of calculations starts up to nodes at distance 1 from $u$, and after they have finished their computations, then $u$ can go on to the next iteration. The time taken by the slowest node at distance $d-1$ from $u$ is distributed as $max \{ X_1,...,X_{(n-\frac{n}{d})}\}$. In general, the time taken by the slowest node at distance $l$ from $u$ is distributed as $max \{ X_1,...,X_{\frac{n}{d}(d-l)}\}$. So the time $u$ have to wait to start iteration $t$ is distributed as 

$$Z = \sum_{l=0}^{d-1} max \{X_1,...,X_{\frac{n}{d}(d-l)}\}$$

$$\mathbb{E}[Z] = \frac{1}{\lambda} \sum_{j=0}^{d-1} \sum_{i=1}^{\frac{n}{d}(d-j)} \frac{1}{i}$$

Clearly this situation is very rare, but it gives the most pessimistic lower bound of the time a node should wait in order to perform one iteration.

**In the following tests the _almost_-lower bound will be used (and not the latter).**


# WTHeck is happening

## What needed a change
Let's consider what is happening inside a node $v$:
- iterations counter starts from 0;
- the weight vector is initialized to a random value (why and the significance of this random value will be discussed later);
- stepwise update is computed:
    - 

- the iteration counter is increased after each update of the local model. For instance, at iteration 0, the first model update is applied, then the iteration counter is set equal to 1. I used to consider iterations counter increase as the last instruction which concludes the update.

The problem arises because metrics calculation is made before the iterations counter increase, hence 
Iteration 0 use to be considered the first update of weight vector applied by each node. The binding between iterations and weights was: iteration $i$ is paired with $w^(i)$. Actually if one thinks at $w^{(0)}$ then it should be the starting point, so $w^{(1)}$ should be the first weight update instead. In my code it wasn't so.

There was `W_log` which stored the history of $w$(s). The first element of `W_log` (e.g. `W_log[0]`) is the starting $w$ which is not the first weight update $w^(0)$


## Why these outcomes?

# 1e-6 alpha 10k samples 10k time GD test
Parameters:
- 100 nodes;
- 10k samples;
- 100 features/predictors;
- plotted graphs: diagonal, cycle, clique;
- 10k time limit;
- **1e-6 alpha**.


## X starts in [0,1]


## X starts in [0,2]


## X starts in 0